In [ ]:
!pip install protobuf==3.20.*
!export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

  Using cached protobuf-3.20.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.4
    Uninstalling protobuf-3.19.4:
      Successfully uninstalled protobuf-3.19.4


In [ ]:
%%capture
import IPython
import sys

!{sys.executable} -m pip install ipywidgets
IPython.Application.instance().kernel.do_shutdown(True)  # has to restart kernel so changes are used

### Installing the necessary libraries

In [ ]:
!pip install transformers
!pip install seqeval
!pip install tensorflow
!pip install torch

  Using cached seqeval-1.2.2-py3-none-any.whl


### Importing the libraries

In [ ]:
import boto3
import os
import shutil
import pandas as pd
import numpy as np

In [ ]:

import torch
import tensorflow
torch.cuda.is_available()
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig,BertForTokenClassification, AdamW,get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split
from tensorflow. keras.preprocessing.sequence import pad_sequences

from seqeval.metrics import f1_score, accuracy_score
from tqdm import tqdm, trange

from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

In [ ]:
print('\nCUDA available?', torch.cuda.is_available())
if torch.cuda.is_available():
    print('CUDA device: ', torch.cuda.current_device())
    print('CUDA device name: ', torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA available? True
CUDA device:  0
CUDA device name:  Tesla T4


In [ ]:
from transformers import RobertaTokenizer, AutoModelForTokenClassification
tokenizer=RobertaTokenizer.from_pretrained("roberta-base")

### Formatting the data for Datasets

In [ ]:
import os,ast
lst=os.listdir('/root/bio_tagged')

In [ ]:
lst=['NERIOBdataV1.jsonl', 'ner_labels_100.csv', 'ner_labels_500.csv', 'ner_new_labels_2286.csv']

In [ ]:
data=pd.DataFrame()
for i in lst:
    if i.endswith('jsonl') or i.endswith('manifest'):
        df=pd.read_json('/root/bio_tagged/'+i,lines=True)
        df=df[['tokens', 'tags']]
        data=data.append(df)
        print(data.shape)
    else:
        df=pd.read_csv('/root/bio_tagged/'+i)
        df['tokens']=df['tokens'].apply(lambda x:ast.literal_eval(x))
        df['tags']=df['tags'].apply(lambda x:ast.literal_eval(x))
        df=df[['tokens', 'tags']]
        data= data.append(df)
        print(data.shape)
data=data[['tokens','tags']]
#data=data.dropna()

<ipython-input-8-232874a3e0d4>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(df)
<ipython-input-8-232874a3e0d4>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data= data.append(df)
<ipython-input-8-232874a3e0d4>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data= data.append(df)


(1584, 2)
(1685, 2)
(2186, 2)
(4471, 2)


<ipython-input-8-232874a3e0d4>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data= data.append(df)


In [ ]:
##labels list
labels_lst=data['tags'].tolist()

In [ ]:
##Keeping unique labels for each entity
final_labels=[]
#lab_lst=[i.lower() for i in lab_lst]
for i in labels_lst:
  x=[]
  for j in i:
    if j!='O':
      p=j[:2]+j[2:].lower()
      p=p.replace('/','_')
      x.append(p)
    else:
      x.append('O')
  final_labels.append(x)

In [ ]:
##Labels to be trained
lab_lst=['issue_problem','feature', 'benefit', 'routine']

In [ ]:
# lab_lst=['brand','product_type','condition','ingredient','time','body_part']

In [ ]:
##Removing minority class labels
final_labels_1=[]
#lab_lst=[i.lower() for i in lab_lst]
for i in final_labels:
  x=[]
  for j in i:
    if j!='O' and j[2:].lower() in lab_lst:
      p=j[:2]+j[2:].lower()
      p=p.replace('/','_')
      x.append(p)
    else:
      x.append('O')
  final_labels_1.append(x)

In [ ]:
data['tags']=final_labels_1

In [ ]:
data.shape

(4471, 2)

In [ ]:
from sklearn.utils import shuffle
data = shuffle(data,random_state=42)

In [ ]:
## Filtering samples without any required entity
data['tags_length']=data['tags'].apply(lambda x: len([i for i in x if i=='O']))
data['token_length']=data['tokens'].apply(lambda x: len(x))
data['flag']=data['tags_length']==data['token_length']
data=data[data['flag']==False]

In [ ]:
data.shape

(1634, 5)

In [ ]:
# tokens level data after Exploding
data['id']=[i for i in range(data.shape[0])]
data=data.explode(['tokens','tags'])


In [ ]:
data['tags'].value_counts()

O                  87882
I-benefit           2131
I-feature           1733
B-benefit           1625
I-issue_problem     1598
B-feature           1063
I-routine            886
B-routine            529
B-issue_problem      412
Name: tags, dtype: int64

In [ ]:
## Transforming original tokens to BERT tokenized form
class GetSentence(object):
    def __init__(self,data):
        self.data = data
        self.n_sentences = 1
        self.empty = False
        agg_function = lambda s: [(w,t) for w,t in zip(s["tokens"].values.tolist(),
                                                        s["tags"].values.tolist())]
        self.group = self.data.groupby('id').apply(agg_function)
        self.sentence = [s for s in self.group]

In [ ]:
getter = GetSentence(data)
sentence = [[word[0] for word in sentence] for sentence in getter.sentence]
labels = [[lab[1] for lab in sentence] for sentence in getter.sentence]
tag_values = list(dict.fromkeys(data["tags"].values))
tag_values.append('PAD')
tag_idx = {'O': 0, 'B-issue_problem': 1, 'I-issue_problem': 2, 'B-feature': 3, 'I-feature': 4, 'B-benefit': 5, 'I-benefit': 6, 'B-routine': 7, 'I-routine': 8, 'PAD': 9}
# tag_idx={'O': 0, 'B-condition': 1, 'B-time': 2, 'I-time': 3, 'B-product_type': 4, 'I-product_type': 5, 'B-brand': 6, 'B-body_part': 7, 'I-brand': 8, 'I-condition': 9, 'B-ingredient': 10, 'I-body_part': 11, 'I-ingredient': 12, 'PAD': 13}

In [ ]:
id2label= {t: i for i, t in tag_idx.items()}
label2id = {i: t for i, t in tag_idx.items()}

In [ ]:
print(label2id)

{'O': 0, 'B-issue_problem': 1, 'I-issue_problem': 2, 'B-feature': 3, 'I-feature': 4, 'B-benefit': 5, 'I-benefit': 6, 'B-routine': 7, 'I-routine': 8, 'PAD': 9}


###### Encoding the data

In [ ]:
MAX_LEN = 512
BATCH_SIZE = 8

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
## Returns tokenized sentence with labels
def tokenize_preserve(sentences,text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentences,text_labels):
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        tokenized_sentence.extend(tokenized_word)
        labels.extend([label]*n_subwords)

    return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [
  tokenize_preserve(sent,labs)
  for sent,labs in zip(sentence,labels)
]

In [ ]:
##Creating new rows of data for samples with token >512
tokenized_texts_and_labels_1=[]
x=0
for i in tokenized_texts_and_labels:
  tokens=i[0]
  labels=i[1]
  if len(tokens)<512:
    tokenized_texts_and_labels_1.append((tokens,labels))
  else:
    cnt=len(tokens)//512+1
    for j in range(0,cnt):
      ind=512*j
      x+=1
      tokenized_texts_and_labels_1.append((tokens[ind:ind+512],labels[ind:ind+512]))



In [ ]:
tokenized_text = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_1]
labels = [token_label_pair[1]for token_label_pair in tokenized_texts_and_labels_1]

In [ ]:
## Generating Input_ids, tags and masks vectors
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_text],
                         maxlen=MAX_LEN, dtype='long',value=0.0,
                         truncating='post',padding='post')
tags = pad_sequences([[tag_idx.get(l)for l in lab]for lab in labels],
                    maxlen=MAX_LEN, dtype='long', value=tag_idx['PAD'],
                    truncating='post',padding='post')
attention_masks = [[float(i !=0.0)for i in ii]for ii in input_ids]

In [ ]:
## Splitting data into train, text and validation
tr_input, test_input,tr_tag, test_tag,tr_masks,test_masks = train_test_split(input_ids,tags,attention_masks,random_state=45,test_size=.2)
tr_input, val_input,tr_tag, val_tag,tr_masks,val_masks = train_test_split(tr_input,tr_tag,tr_masks,random_state=45,test_size=.2)

In [ ]:
tr_input.shape,val_input.shape,tr_tag.shape,test_input.shape

((1056, 512), (264, 512), (1056, 512), (331, 512))

### Adding the data to the Dataset

In [ ]:
tr_input = torch.tensor(tr_input)
val_input = torch.tensor(val_input)
tr_tag = torch.tensor(tr_tag)
val_tag = torch.tensor(val_tag)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

test_input = torch.tensor(test_input)
test_tag = torch.tensor(test_tag)
test_masks = torch.tensor(test_masks)


In [ ]:
train_data = TensorDataset(tr_input, tr_masks, tr_tag)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

valid_data = TensorDataset(val_input, val_masks,val_tag)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=BATCH_SIZE)

In [ ]:

test_data = TensorDataset(test_input, test_masks,test_tag)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

## Loading the model

In [ ]:

from transformers import AutoModelForTokenClassification,AutoConfig, AutoModel, AutoTokenizer, AdamW, get_cosine_schedule_with_warmup
model = AutoModelForTokenClassification.from_pretrained("roberta-base",num_labels=len(label2id),ignore_mismatched_sizes=True)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tensorflow.keras.optimizers import Adamax,AdamW

In [ ]:
###### Defining the training arguments
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = torch.optim.AdamW(
    optimizer_grouped_parameters,
    lr=4e-5,
    eps=1e-8
)


epochs = 40
max_grad_norm = 1

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
model.cuda()

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm

In [ ]:
## Training of NER
from time import time
start = time()
loss_values, validation_loss_values = [], []
cnt=0
epch=0

for _ in trange(epochs, desc= "Epoch"):
    if cnt==0:
    #/|\==>TRAINLOOP(ONEPASS)<==\|/
        model.train()
        total_loss=0 #so it resets each epoch

        for step, batch in enumerate(train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch #also the order in train_data/val_data

            model.zero_grad() #clearing previous gradients for each epoch

            outputs = model(b_input_ids,
                           attention_mask=b_input_mask, labels=b_labels)#forward pass

            loss = outputs[0]
            loss.backward() #getting the loss and performing backward pass

            total_loss += loss.item() #tracking loss

            torch.nn.utils.clip_grad_norm(parameters=model.parameters(), max_norm=max_grad_norm)
            #^^^ preventing exploding grads

            optimizer.step() #updates parameters

            scheduler.step() #update learning_rate

        avg_train_loss = total_loss/len(train_dataloader)
        print('Average train loss : {}'.format(avg_train_loss))

        loss_values.append(avg_train_loss) #storing loss values if you choose to plot learning curve

        #/|\==>VALIDATION(ONEPASS)<==\|/
        model.eval()

        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        predictions, true_labels = [], []

        for batch in valid_dataloader:
            batch = tuple(t.to(device)for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            with torch.no_grad():
                outputs = model(b_input_ids,
                               attention_mask=b_input_mask, labels=b_labels)

            logits = outputs[1].detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            eval_loss += outputs[0].mean().item()
            predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
            true_labels.extend(label_ids)
        eval_loss = eval_loss / len(valid_dataloader)
        validation_loss_values.append(eval_loss)
        print('Validation loss: {}'.format(eval_loss))

        pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                    for p_i, l_i, in zip(p,l)if tag_values[l_i] !='PAD']

        valid_tags = [tag_values[l_i]for l in true_labels
                     for l_i in l if tag_values[l_i] !='PAD']

        print('Validation Accuracy: {}'.format(accuracy_score(pred_tags,valid_tags)))

        print('Validation F-1 Score:{}'.format(f1_score([pred_tags], [valid_tags])))

        model.save_pretrained('/root/bio_tagged/models/epoch_'+str(epch))



print(f'Time taken to run: {time() - start} seconds')

In [ ]:
## Loading trained model
model = AutoModelForTokenClassification.from_pretrained("/root/bio_tagged/models/epoch_20",num_labels=len(label2id),ignore_mismatched_sizes=True)

In [ ]:
model.cuda()

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm

In [ ]:
##Validation Data
predictions, true_labels = [], []

for batch in valid_dataloader:
    batch = tuple(t.to(device)for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids,
                       attention_mask=b_input_mask, labels=b_labels)

    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)
pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
            for p_i, l_i, in zip(p,l)if tag_values[l_i] !='PAD']

valid_tags = [tag_values[l_i]for l in true_labels
             for l_i in l if tag_values[l_i] !='PAD']

display(print(classification_report([pred_tags], [valid_tags])))

               precision    recall  f1-score   support

      benefit       0.93      0.90      0.92       288
      feature       0.96      0.93      0.94       115
issue_problem       0.93      0.97      0.95       601
      routine       0.96      0.95      0.95       560

    micro avg       0.94      0.95      0.94      1564
    macro avg       0.94      0.94      0.94      1564
 weighted avg       0.94      0.95      0.94      1564



None

In [ ]:
##Test data
predictions, true_labels = [], []

for batch in test_dataloader:
    batch = tuple(t.to(device)for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids,
                       attention_mask=b_input_mask, labels=b_labels)

    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)
pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
            for p_i, l_i, in zip(p,l)if tag_values[l_i] !='PAD']

test_tags = [tag_values[l_i]for l in true_labels
             for l_i in l if tag_values[l_i] !='PAD']

display(print(classification_report([pred_tags], [test_tags])))

               precision    recall  f1-score   support

      benefit       0.91      0.87      0.89       436
      feature       0.81      0.89      0.85        97
issue_problem       0.87      0.94      0.91       658
      routine       0.95      0.85      0.89       753

    micro avg       0.90      0.89      0.89      1944
    macro avg       0.88      0.89      0.88      1944
 weighted avg       0.91      0.89      0.89      1944



None